In [1400]:
from collections import OrderedDict
from conllu import parse
from enum import Enum
import numpy as np
import json
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
import tokenize_uk
import pymorphy2
import stanza
import spacy
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.layers.core import Dense
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dropout, Activation
from keras.layers import Flatten
from keras.utils import to_categorical
from conllu import parse
from gensim.models.word2vec import Word2Vec, LineSentence
from gensim.models import KeyedVectors
from enum import Enum
from collections import OrderedDict
import progressbar
import tensorflow as tf
import random as python_random

In [1401]:
PATH = '../../../../UD_Ukrainian-IU'

with open(PATH + '/uk_iu-ud-train.conllu') as f:
    train_data = f.read()
    
with open(PATH + '/uk_iu-ud-dev.conllu') as f:
    test_data = f.read()

train_trees = parse(train_data)
test_trees = parse(test_data)

In [1402]:
nlp_uk = spacy.load('/Users/lissm/Work/Dev/NLP/corpora/vectors/ubercorpus_lowercased_tokenized')

In [1403]:
# class Actions(str, Enum):
#     SHIFT = 'shift'
#     REDUCE = 'reduce'
#     RIGHT = 'right'
#     LEFT = 'left'
    
class Actions(int, Enum):
    SHIFT = 0
    REDUCE = 1
    RIGHT = 2
    LEFT = 3


ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats',
                                        None), ('head', None),  ('deprel', None),
                    ('deps', None), ('misc', None)])


# def dep_parse(tree, clf, vectorizer, feature_extractor):
#     stack, queue, relations = [ROOT], tree[:], []

#     while queue or stack:
#         if stack and not queue:
#             stack.pop()
#         else:
#             features = feature_extractor(stack, queue, relations)
#             action = clf.predict(vectorizer.transform([features]))[0]

#             if action == Actions.SHIFT:
#                 stack.append(queue.pop(0))
#             elif action == Actions.REDUCE:
#                 stack.pop()
#             elif action == Actions.LEFT:
#                 deprel = stack[-1]['deprel'] or predict_deprel(
#                     stack[-1], queue[0], stack + queue)
#                 rel = (stack[-1]['id'], queue[0]['id'], deprel)
#                 relations.append(rel)
#                 stack.pop()
#             elif action == Actions.RIGHT:
#                 deprel = queue[0]['deprel'] or predict_deprel(
#                     queue[0], stack[-1], stack + queue)
#                 rel = (queue[0]['id'], stack[-1]['id'], deprel)
#                 relations.append(rel)
#                 stack.append(queue.pop(0))

#     return sorted(relations)

In [6]:
""" Data & reporting utils """


def calculate_as(trees, clf, vect, feature_extractor):
    total, tpu, tpl, full_match = 0, 0, 0, 0
    golden_u, golden_l = None, None
    for tree in trees:
        tree = [t for t in tree if type(t['id']) == int]
        golden_all = [(node['id'], node['head'], node['deprel'])
                      for node in tree]
        golden_u = [(x, y) for x, y, _ in golden_all]

        predicted_all = dep_parse(tree, clf, vect, feature_extractor)
        predicted_u = [(x, y) for x, y, _ in predicted_all]

        total += len(tree)
        tpu += len(set(golden_u).intersection(set(predicted_u)))
        tpl += len(set(golden_all).intersection(set(predicted_all)))

        if set(golden_all) == set(predicted_all):
            full_match += 1

    print('== Attachment score report ==')
    print('Total: ', total)
    print('Match unlabeled: ', tpu)
#     print('Match labeled: ', tpl)
    print('UAS: ', round(tpu/total, 2))
#     print('LAS: ', round(tpl/total, 2))
    print("Full match:", round(full_match/len(trees), 2))


def get_lrc_classifier():
    pipe = Pipeline([
        ('dict_vect', DictVectorizer()),
        ('lrc', LogisticRegression(random_state=42, multi_class='multinomial',
                                   max_iter=100, solver='sag', n_jobs=20))])

    return pipe


def print_result(train_trees, test_trees, clf, feature_extractor):
    train_feat, train_lab = get_data(train_trees, feature_extractor)
    test_feat, test_lab = get_data(test_trees, feature_extractor)

    clf.fit(train_feat, train_lab)
    print(classification_report(test_lab, clf.predict(test_feat)))
    calculate_as(test_trees, clf['lrc'], clf['dict_vect'], feature_extractor)

In [1536]:
def get_w2v_model(trees):
    words = []
    for tree in trees:
        words.append([x['lemma'] for x in tree])
    model = Word2Vec(
        words, size=300, min_count=1, workers=6,
        window=5, alpha=0.030, negative=7, iter=5
    )

    return model


def get_pos_tag_map(trees):
    pos_map = {'ROOT': 0, 'UNK': 1}
    cnt = 1

    for tree in trees:
        for tok in tree:
            if not pos_map.get(tok['upostag']):
                pos_map[tok['upostag']] = cnt
                cnt += 1
    return pos_map
    

def oracle(stack, top_queue, relations):
    top_stack = stack[-1]
    if top_stack and not top_queue:
        return Actions.REDUCE
    elif top_queue['head'] == top_stack['id']:
        return Actions.RIGHT
    elif top_stack['head'] == top_queue['id']:
        return Actions.LEFT
    elif top_stack['id'] in [i[0] for i in relations] and \
        (top_queue['head'] < top_stack['id'] or
         [s for s in stack if s['head'] == top_queue['id']]):
        return Actions.REDUCE
    else:
        return Actions.SHIFT


def get_data_for_tree(tree, pos_map):
    res = []
    stack, buffer, relations = [ROOT], tree[:], []
    top_feats = 0
    
    while buffer or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        buffer[0] if len(buffer) > 0 else None,
                        relations)
        # we need 3 words from stack and buffer to train the LSTM model
        if top_feats < 3 and len(stack) and len(buffer):
            top_stack = stack[-1]
            top_buff = buffer[0]
            res.append((top_stack['lemma'], pos_map[top_stack['upostag'] or pos_map['UNK']],
                        top_buff['lemma'], pos_map[top_buff['upostag'] or pos_map['UNK']],
                        action.value))
            top_feats += 1
        if action == Actions.SHIFT:
            stack.append(buffer.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            rel = (stack[-1]['id'], buffer[0]['id'])
            relations.append(rel)
            stack.pop()
        elif action == Actions.RIGHT:
            rel = (buffer[0]['id'], stack[-1]['id'])
            relations.append(rel)
            stack.append(buffer.pop(0))

    return res


def get_data(trees, v2w_model, pos_map):
    vectors, labels = [], []
    for tree in trees:
        t_vectors, t_labels = [], []
        tree_data = get_data_for_tree(
            [t for t in tree if type(t['id']) == int], pos_map)
        for stack, stack_pos, buff, buff_pos, label in tree_data:
            try:
                stack_vect = v2w_model.wv[stack]
                buff_vect = v2w_model.wv[buff]
                stack_vect_pos = np.append(stack_vect, [stack_pos])
                buff_vect_pos = np.append(buff_vect, [buff_pos])
                vc = np.concatenate((stack_vect_pos, buff_vect_pos))
                vectors.append(vc)
                labels.append(label)
            except Exception as e:
                pass

    return np.dstack(vectors), np.dstack(labels)


def get_predicted_label(labels):
    l = labels[0][0]
    return np.argmax(l)


def get_target_data(pairs, v2w_model):
    vectors = []
    for stack, buffer in pairs:
        try:
            stack_vect = v2w_model.wv[stack]
            buff_vect = v2w_model.wv[buff]
            vectors.append(np.concatenate((stack_vect, buff_vect)))
        except:
            pass

    return np.dstack(vectors)


def get_data_with_timesteps(data, steps):
    res = []
    for i, x in enumerate(data):
        a = []
        a.append(data[i][0])
        for j in range(steps - 1):
            if i + j < len(data) - 1:
                a.append(data[i+j][0])
            else:
                a.append(data[i][0])
        res.append(np.array(a))
    return np.array(res)

In [1535]:
def make_model(n_vocab, n_classes, X):
    model = Sequential()
    model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(X.shape[1], X.shape[2])))
    model.add(LSTM(512, return_sequences=True, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())

    return model


def to_cat(y_int, vocab_size):
    return to_categorical(y_int, num_classes=vocab_size)


def train_model(model, epochs, x, y, n_classes):
    model.fit(x, y, epochs=epochs, verbose=1)

    
def predict(model, x):
    return model.predict(x)

In [1406]:
np.random.seed(42)
python_random.seed(42)
tf.random.set_seed(42)

In [1407]:
v2w_model = get_w2v_model(train_trees)

2020-05-26 00:39:11 WARNING: under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


In [1442]:
pos_map = get_pos_tag_map(train_trees)

In [1540]:
train_x, train_y = get_data(train_trees, v2w_model, pos_map)
n_vocab = len(train_x)
n_classes = 4
x = np.moveaxis(train_x, -1, 0)
y = np.moveaxis(train_y, -1, 0)
y = np.array([to_cat(x, n_classes) for x in y])
x = get_data_with_timesteps(x, 20)
y = get_data_with_timesteps(y, 20)

In [1544]:
model = make_model(n_vocab, n_classes, x)

Model: "sequential_135"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_212 (LSTM)              (None, 20, 256)           879616    
_________________________________________________________________
lstm_213 (LSTM)              (None, 20, 512)           1574912   
_________________________________________________________________
dense_147 (Dense)            (None, 20, 64)            32832     
_________________________________________________________________
dropout_72 (Dropout)         (None, 20, 64)            0         
_________________________________________________________________
dense_148 (Dense)            (None, 20, 4)             260       
Total params: 2,487,620
Trainable params: 2,487,620
Non-trainable params: 0
_________________________________________________________________
None


In [1545]:
train_model(model, 100, x, y, n_classes)

Epoch 1/100
8804/8804 [==============================] - 67s 8ms/step - loss: 0.8726 - accuracy: 0.6474
Epoch 2/100
8804/8804 [==============================] - 59s 7ms/step - loss: 0.7005 - accuracy: 0.7139
Epoch 3/100
8804/8804 [==============================] - 61s 7ms/step - loss: 0.4888 - accuracy: 0.7982
Epoch 4/100
8804/8804 [==============================] - 70s 8ms/step - loss: 0.2744 - accuracy: 0.8941
Epoch 5/100
8804/8804 [==============================] - 63s 7ms/step - loss: 0.1870 - accuracy: 0.9292
Epoch 6/100
8804/8804 [==============================] - 67s 8ms/step - loss: 0.1607 - accuracy: 0.9376
Epoch 7/100
8804/8804 [==============================] - 65s 7ms/step - loss: 0.1475 - accuracy: 0.9432
Epoch 8/100
8804/8804 [==============================] - 66s 7ms/step - loss: 0.1355 - accuracy: 0.9470
Epoch 9/100
8804/8804 [==============================] - 65s 7ms/step - loss: 0.1321 - accuracy: 0.9485
Epoch 10/100
8804/8804 [==============================] - 60s 7m

8804/8804 [==============================] - 61s 7ms/step - loss: 0.0522 - accuracy: 0.9804
Epoch 80/100
8804/8804 [==============================] - 61s 7ms/step - loss: 0.0501 - accuracy: 0.9809
Epoch 81/100
8804/8804 [==============================] - 61s 7ms/step - loss: 0.0517 - accuracy: 0.9804
Epoch 82/100
8804/8804 [==============================] - 62s 7ms/step - loss: 0.0518 - accuracy: 0.9800
Epoch 83/100
8804/8804 [==============================] - 61s 7ms/step - loss: 0.0545 - accuracy: 0.9793
Epoch 84/100
8804/8804 [==============================] - 61s 7ms/step - loss: 0.0639 - accuracy: 0.9766
Epoch 85/100
8804/8804 [==============================] - 61s 7ms/step - loss: 0.0488 - accuracy: 0.9813
Epoch 86/100
8804/8804 [==============================] - 62s 7ms/step - loss: 0.0468 - accuracy: 0.9817
Epoch 87/100
8804/8804 [==============================] - 61s 7ms/step - loss: 0.0457 - accuracy: 0.9822
Epoch 88/100
8804/8804 [==============================] - 61s 7ms/st

In [1546]:
model.save('oracle.h5')